Task from Asan.

In [1]:
import os
import shutil
import SimpleITK as sitk
import nilearn.image
import nilearn.plotting
import nibabel as nib
import numpy as np
from glob import glob

In [2]:
def make_dirs(path_lst):
    for path in path_lst:
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            pass

# make outputdir and return output path (including filename) by combining outDir, pt, filename
def get_outpath(outDir, pt, filename):
    outPath = os.path.join(outDir, pt, filename)

    make_dirs([os.path.join(outDir, pt)])

    if os.path.exists(outPath):
        print('{} already exists'.format(filename))
        return None
    else:
        return outPath

# find sequence from filename
def find_seq(filename):
    seq_lst = ["T1_GD", "T1GD", "T1CE", "T1C", "T1", "T2", "FLAIR", "F", "FLAIRGD", "ADC", "BB", "WB"]
    seq = None
    for i in seq_lst:
        if i in filename.upper():
            seq = i
            return seq
    if seq is None:
        print("No matching sequence")
        return None

In [8]:
# 0) flip: orientataion LPS
source = glob('/media/sc-mlsg/SAMSUNG/Meningioma_preprocessed/Asan/Registered/**/*nii*')

# reorient both image and header
total = 0
lst = []
for filepath in source:
    total+=1
    # load img
    img = nib.load(filepath)
    
    # if the image orientation is not LPS
    if nib.aff2axcodes(img.affine) != ('L', 'P', 'S'):
        print(total, filepath.split(os.path.sep)[-1])
        # print orientation of original img
        print(nib.aff2axcodes(img.affine))
        sitk_img = sitk.ReadImage(filepath)
        print(np.array(sitk_img.GetDirection()).reshape(3,3))
        if nib.aff2axcodes(img.affine)[0] == 'R':
            # reorient img(use np.flip or np.transpose)
            img_data = sitk.GetArrayFromImage(sitk_img)
            img_T = np.flip(img_data, axis=2)

            # reset image header info
            flipFilter = sitk.FlipImageFilter()

            flipAxes = (True, False, False) #(width, height, depth)
            flipFilter.SetFlipAxes(flipAxes)

            sitk_img = flipFilter.Execute(sitk_img)

        
        if nib.aff2axcodes(img.affine)[1] == 'A':
            # reorient img(use np.flip or np.transpose)
            img_data = sitk.GetArrayFromImage(sitk_img)
            img_T = np.flip(img_data, axis=1)

            # reset image header info
            flipFilter = sitk.FlipImageFilter()

            flipAxes = (False, True, False) #(width, height, depth)
            flipFilter.SetFlipAxes(flipAxes)

            sitk_img = flipFilter.Execute(sitk_img)

        if nib.aff2axcodes(img.affine)[2] == 'I':
            # reorient img(use np.flip or np.transpose)
            img_data = sitk.GetArrayFromImage(sitk_img)
            img_T = np.flip(img_data, axis=0)

            # reset image header info
            flipFilter = sitk.FlipImageFilter()

            flipAxes = (False, False, True) #(width, height, depth)
            flipFilter.SetFlipAxes(flipAxes)

            sitk_img = flipFilter.Execute(sitk_img)
        
        # save new header info
        ni_img = sitk.GetImageFromArray(img_T)
        ni_img.SetOrigin(sitk_img.GetOrigin())
        ni_img.SetSpacing(sitk_img.GetSpacing())
        ni_img.SetDirection(sitk_img.GetDirection())
        
        # save file
        outpath = os.path.join(filepath)
        
        sitk.WriteImage(ni_img, outpath)
        
        # check orientation of reoriented img
        img = nib.load(outpath)
        print(nib.aff2axcodes(img.affine))
        
        sitk_mask = sitk.ReadImage(outpath)
        print(np.array(sitk_mask.GetDirection()).reshape(3,3))
        

1 001_T1C-label_resize.nii
('L', 'P', 'I')
[[ 0.99931322 -0.0343474   0.01390479]
 [ 0.0348706   0.99861811 -0.0393182 ]
 [ 0.0125351  -0.03977606 -0.99912999]]
('L', 'P', 'S')
[[ 0.99931322 -0.0343474  -0.01390479]
 [ 0.0348706   0.99861811  0.0393182 ]
 [ 0.0125351  -0.03977606  0.99912999]]
2 001_T1C_registered.nii
('L', 'P', 'I')
[[ 0.99931322 -0.0343474   0.01390479]
 [ 0.0348706   0.99861811 -0.0393182 ]
 [ 0.0125351  -0.03977606 -0.99912999]]
('L', 'P', 'S')
[[ 0.99931322 -0.0343474  -0.01390479]
 [ 0.0348706   0.99861811  0.0393182 ]
 [ 0.0125351  -0.03977606  0.99912999]]
3 001_T2_registered.nii
('L', 'P', 'I')
[[ 0.99931322 -0.0343474   0.01390479]
 [ 0.0348706   0.99861811 -0.0393182 ]
 [ 0.0125351  -0.03977606 -0.99912999]]
('L', 'P', 'S')
[[ 0.99931322 -0.0343474  -0.01390479]
 [ 0.0348706   0.99861811  0.0393182 ]
 [ 0.0125351  -0.03977606  0.99912999]]
4 002_T1C-label_resize.nii
('L', 'P', 'I')
[[ 9.99262872e-01  2.31550826e-03 -3.83190655e-02]
 [-3.88650989e-05  9.98239

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
41 020_T1C_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
42 020_T2_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
43 021_T1C-label_resize.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
44 021_T1C_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
45 021_T2_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
46 022_T1C-label_resize.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
47 022_T1C_registered.nii
('L', 'P', 'I')
[[ 1.  0.  

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
107 061_T1C_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
108 061_T2_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
109 064_T1C-label_resize.nii
('L', 'P', 'I')
[[ 9.99657417e-01 -1.57395580e-03  2.61260622e-02]
 [-1.15298247e-04  9.97915714e-01  6.45307296e-02]
 [ 2.61731787e-02  6.45116362e-02 -9.97573663e-01]]
('L', 'P', 'S')
[[ 9.99657417e-01 -1.57395580e-03 -2.61260622e-02]
 [-1.15298247e-04  9.97915714e-01 -6.45307296e-02]
 [ 2.61731787e-02  6.45116362e-02  9.97573663e-01]]
110 064_T1C_registered.nii
('L', 'P', 'I')
[[ 9.99657417e-01 -1.57395580e-03  2.61260622e-02]
 [-1.15298247e-04  9.97915714e-01  6.45307296e-02]
 [ 2.61731787e-02  6.45116362e-02 -9.97573663e-01]]
('L', 'P', 'S')
[[ 9.99657417e-01 -1.57395580e-03 -2

('L', 'P', 'S')
[[ 1.00000000e+00  4.54323254e-10 -1.57718836e-08]
 [ 4.54323246e-10  9.98341817e-01  5.75640286e-02]
 [ 1.57718834e-08 -5.75640286e-02  9.98341817e-01]]
146 091_T1C_registered.nii
('L', 'P', 'I')
[[ 1.00000000e+00  4.54323254e-10  1.57718836e-08]
 [ 4.54323246e-10  9.98341817e-01 -5.75640286e-02]
 [ 1.57718834e-08 -5.75640286e-02 -9.98341817e-01]]
('L', 'P', 'S')
[[ 1.00000000e+00  4.54323254e-10 -1.57718836e-08]
 [ 4.54323246e-10  9.98341817e-01  5.75640286e-02]
 [ 1.57718834e-08 -5.75640286e-02  9.98341817e-01]]
147 091_T2_registered.nii
('L', 'P', 'I')
[[ 1.00000000e+00  4.54323254e-10  1.57718836e-08]
 [ 4.54323246e-10  9.98341817e-01 -5.75640286e-02]
 [ 1.57718834e-08 -5.75640286e-02 -9.98341817e-01]]
('L', 'P', 'S')
[[ 1.00000000e+00  4.54323254e-10 -1.57718836e-08]
 [ 4.54323246e-10  9.98341817e-01  5.75640286e-02]
 [ 1.57718834e-08 -5.75640286e-02  9.98341817e-01]]
148 092_T1C-label_resize.nii
('L', 'P', 'I')
[[ 9.97052715e-01 -6.56283497e-03 -7.64382868e-02]
 

('L', 'P', 'S')
[[ 9.99610094e-01  1.21786687e-04 -2.79221427e-02]
 [ 1.21786692e-04  9.99961960e-01  8.72143930e-03]
 [ 2.79221431e-02 -8.72143906e-03  9.99572054e-01]]
172 105_T1C-label_resize.nii
('L', 'P', 'I')
[[ 9.99847703e-01 -1.52177586e-05  1.74519666e-02]
 [-1.52177580e-05  9.99998479e-01  1.74382519e-03]
 [ 1.74519661e-02  1.74382520e-03 -9.99846182e-01]]
('L', 'P', 'S')
[[ 9.99847703e-01 -1.52177586e-05 -1.74519666e-02]
 [-1.52177580e-05  9.99998479e-01 -1.74382519e-03]
 [ 1.74519661e-02  1.74382520e-03  9.99846182e-01]]
173 105_T1C_registered.nii
('L', 'P', 'I')
[[ 9.99847703e-01 -1.52177586e-05  1.74519666e-02]
 [-1.52177580e-05  9.99998479e-01  1.74382519e-03]
 [ 1.74519661e-02  1.74382520e-03 -9.99846182e-01]]
('L', 'P', 'S')
[[ 9.99847703e-01 -1.52177586e-05 -1.74519666e-02]
 [-1.52177580e-05  9.99998479e-01 -1.74382519e-03]
 [ 1.74519661e-02  1.74382520e-03  9.99846182e-01]]
174 105_T2_registered.nii
('L', 'P', 'I')
[[ 9.99847703e-01 -1.52177586e-05  1.74519666e-02]
 

('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
222 139_T2_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
223 141_T1C-label_resize.nii
('L', 'P', 'I')
[[ 9.98721034e-01 -2.50524768e-03 -5.04977230e-02]
 [ 3.46647529e-05  9.98805331e-01 -4.88662507e-02]
 [-5.05598136e-02 -4.88019989e-02 -9.97527979e-01]]
('L', 'P', 'S')
[[ 9.98721034e-01 -2.50524768e-03  5.04977230e-02]
 [ 3.46647529e-05  9.98805331e-01  4.88662507e-02]
 [-5.05598136e-02 -4.88019989e-02  9.97527979e-01]]
224 141_T1C_registered.nii
('L', 'P', 'I')
[[ 9.98721034e-01 -2.50524768e-03 -5.04977230e-02]
 [ 3.46647529e-05  9.98805331e-01 -4.88662507e-02]
 [-5.05598136e-02 -4.88019989e-02 -9.97527979e-01]]
('L', 'P', 'S')
[[ 9.98721034e-01 -2.50524768e-03  5.04977230e-02]
 [ 3.46647529e-05  9.98805331e-01  4.88662507e-02]
 [-5.05598136e-02 -4.88019989e-02  9.97527979e-01]]
225 141_T2_registered.nii
('L', 'P', 'I')
[[ 9.98721034e-01 -2.5052476

('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
262 202_T1C-label_resize.nii
('L', 'P', 'I')
[[ 9.97185049e-01  1.30511540e-03  7.49684889e-02]
 [-1.27402134e-04  9.99876550e-01 -1.57120791e-02]
 [ 7.49797372e-02 -1.56582984e-02 -9.97062113e-01]]
('L', 'P', 'S')
[[ 9.97185049e-01  1.30511540e-03 -7.49684889e-02]
 [-1.27402134e-04  9.99876550e-01  1.57120791e-02]
 [ 7.49797372e-02 -1.56582984e-02  9.97062113e-01]]
263 202_T1C_registered.nii
('L', 'P', 'I')
[[ 9.97185049e-01  1.30511540e-03  7.49684889e-02]
 [-1.27402134e-04  9.99876550e-01 -1.57120791e-02]
 [ 7.49797372e-02 -1.56582984e-02 -9.97062113e-01]]
('L', 'P', 'S')
[[ 9.97185049e-01  1.30511540e-03 -7.49684889e-02]
 [-1.27402134e-04  9.99876550e-01  1.57120791e-02]
 [ 7.49797372e-02 -1.56582984e-02  9.97062113e-01]]
264 202_T2_registered.nii
('L', 'P', 'I')
[[ 9.97185049e-01  1.30511540e-03  7.49684889e-02]
 [-1.27402134e-04  9.99876550e-01 -1.57120791e-02]
 [ 7.49797372e-02 -1.56582984e-02 -9.97062113e-01]]
('L', 'P', 'S')

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
296 214_T1C_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
297 214_T2_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
298 215_T1C-label_resize.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
299 215_T1C_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
300 215_T2_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
301 217_T1C-label_resize.nii
('L', 'P', 'I')
[[ 9.99876634e-01 -5.48284303e-05 -1.57071114e-02]
 [-5.48284306e-05  9.99975632e-01 -6.98083611e-03]
 [-1.57071112e-02 -6.98083600e-03 -9.9985226

[[ 1.00000000e+00  5.17241479e-10  2.57667917e-08]
 [ 5.17241472e-10  9.99194397e-01 -4.01317445e-02]
 [ 2.57667914e-08 -4.01317445e-02 -9.99194397e-01]]
('L', 'P', 'S')
[[ 1.00000000e+00  5.17241423e-10 -2.57667899e-08]
 [ 5.17241416e-10  9.99194397e-01  4.01317445e-02]
 [ 2.57667896e-08 -4.01317445e-02  9.99194397e-01]]
330 227_T2_registered.nii
('L', 'P', 'I')
[[ 1.00000000e+00  5.17241423e-10  2.57667899e-08]
 [ 5.17241416e-10  9.99194397e-01 -4.01317445e-02]
 [ 2.57667896e-08 -4.01317445e-02 -9.99194397e-01]]
('L', 'P', 'S')
[[ 1.00000000e+00  5.17241423e-10 -2.57667882e-08]
 [ 5.17241416e-10  9.99194397e-01  4.01317445e-02]
 [ 2.57667878e-08 -4.01317445e-02  9.99194397e-01]]
331 228_T1C-label_resize.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
332 228_T1C_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
333 228_T2_registered.ni

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
374 124_T1C_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
375 124_T2_registered.nii
('L', 'P', 'I')
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
('L', 'P', 'S')
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
376 203_T1C-label_resize.nii
('L', 'P', 'I')
[[ 9.98134740e-01 -6.10494874e-02  6.10905806e-08]
 [ 6.08171826e-02  9.94336544e-01 -8.71556464e-02]
 [-5.32074696e-03 -8.69930833e-02 -9.96194707e-01]]
('L', 'P', 'S')
[[ 9.98134740e-01 -6.10494874e-02 -6.10905806e-08]
 [ 6.08171826e-02  9.94336544e-01  8.71556464e-02]
 [-5.32074696e-03 -8.69930833e-02  9.96194707e-01]]
377 203_T1C_registered.nii
('L', 'P', 'I')
[[ 9.98134740e-01 -6.10494874e-02  6.10905806e-08]
 [ 6.08171826e-02  9.94336544e-01 -8.71556464e-02]
 [-5.32074696e-03 -8.69930833e-02 -9.96194707e-01]]
('L', 'P', 'S')
[[ 9.98134740e-01 -6.10494874e-02 -6

In [25]:
sourceDir = '/media/sc-mlsg/SAMSUNG/Meningioma_preprocessed/Asan/00_Registered'
targetDir = '/media/sc-mlsg/SAMSUNG/Meningioma_preprocessed/Asan/'
maskDir = None
outputPath = '/home/scmlsg/MONSTR/output'

croppedDir = os.path.join(targetDir, '01_Crop_n_padded')
brainmaskDir = os.path.join(targetDir, 'BrainMask')
strippedDir = os.path.join(targetDir, '01_Stripped_wo_Normalized')
znormedDir = os.path.join(targetDir, '02_Stripped_znormalized')
paddedDir = os.path.join(targetDir, '03_Crop_n_padded')

make_dirs([targetDir, croppedDir, brainmaskDir, strippedDir, znormedDir])

T1C_label = 'T1C' # 'BB','T1C','T1GD' etc.
mask_label = 'mask' # '_mask', '-label' etc.

In [19]:
# 1) Skull stripping using MONSTR

# we'll work in the MONSTR directory
workingDir = '/home/sc-mlsg/MONSTR'
%cd {workingDir} 
outputPath = '/home/sc-mlsg/MONSTR/output'

total = 0

for (path, dirs, files) in os.walk(sourceDir):
    if files != []:
        total += 1
        print("skull stripping: #{}, {}".format(total, files))

        pt = path.split(os.path.sep)[-1]
        T1C_path = glob(os.path.join(path, '*{}*'.format(T1C_label)))[1]
        try:
            T2_path = glob(os.path.join(path, '*T2*'))[0]
        except:
            print("T2 not exists")
            T2_path = None
            
        newFileName = pt + '_brain_mask.nii.gz'
        if os.path.exists(os.path.join(brainmaskDir, newFileName)):
            print("{} already exists".format(newFileName))
            continue

        !./MONSTR.sh --t1 {T1C_path} --t2 {T2_path} --o output --atlasdir TBI_atlas --reg

        createdDir = os.path.join(outputPath, os.listdir(outputPath)[0])

        for file in os.listdir(createdDir):
            if file == 'atlas4_brainmask_ANTS.nii.gz':
                shutil.copyfile(os.path.join(createdDir, file), os.path.join(brainmaskDir, newFileName))

        shutil.rmtree(createdDir)

/home/sc-mlsg/MONSTR
skull stripping: #1, ['001_T1C-label_resize.nii', '001_T1C_registered.nii', '001_T2_registered.nii']
001_brain_mask.nii.gz already exists
skull stripping: #2, ['002_T1C-label_resize.nii', '002_T1C_registered.nii', '002_T2_registered.nii']
002_brain_mask.nii.gz already exists
skull stripping: #3, ['003_T1C-label_resize.nii', '003_T1C_registered.nii', '003_T2_registered.nii']
003_brain_mask.nii.gz already exists
skull stripping: #4, ['006_T1C-label_resize.nii', '006_T1C_registered.nii', '006_T2_registered.nii']
006_brain_mask.nii.gz already exists
skull stripping: #5, ['008_T1C-label_resize.nii', '008_T1C_registered.nii', '008_T2_registered.nii']
008_brain_mask.nii.gz already exists
skull stripping: #6, ['009_T1C-label_resize.nii', '009_T1C_registered.nii', '009_T2_registered.nii']
009_brain_mask.nii.gz already exists
skull stripping: #7, ['010_T1C-label_resize.nii', '010_T1C_registered.nii', '010_T2_registered.nii']
010_brain_mask.nii.gz already exists
skull strippi

119_brain_mask.nii.gz already exists
skull stripping: #66, ['121_T1C-label_resize.nii', '121_T1C_registered.nii', '121_T2_registered.nii']
121_brain_mask.nii.gz already exists
skull stripping: #67, ['125_T1C-label_resize.nii', '125_T1C_registered.nii', '125_T2_registered.nii']
125_brain_mask.nii.gz already exists
skull stripping: #68, ['126_T1C-label_resize.nii', '126_T1C_registered.nii', '126_T2_registered.nii']
126_brain_mask.nii.gz already exists
skull stripping: #69, ['127_T1C-label_resize.nii', '127_T1C_registered.nii', '127_T2_registered.nii']
127_brain_mask.nii.gz already exists
skull stripping: #70, ['128_T1C-label_resize.nii', '128_T1C_registered.nii', '128_T2_registered.nii']
128_brain_mask.nii.gz already exists
skull stripping: #71, ['129_T1C-label_resize.nii', '129_T1C_registered.nii', '129_T2_registered.nii']
I found ANTs installation at /opt/ANTs/bin 
Default number of atlases: 4 
Number of processors is not mentioned. Using all available 6 processors. 
Atlas directory : 

Output warped image: atlas3_brainmask_ANTS.nii
AntsExample.sh t1.nii /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii fastfortesting atlas4_T1_ANTS.nii 6
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image: atlas4_T1_ANTS.nii
ANTS deformable registration took 0 HRS 2 MIN 52 SEC 
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T2.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel v

invalid input dataset <t1_MultiModalStripMask_smooth.nii>
atlas1_brainmask_ANTS.nii:	 99.8% -- replaced with atlas1_brainmask_ANTS.nii.gz
atlas1_T1_ANTS.nii:	 36.8% -- replaced with atlas1_T1_ANTS.nii.gz
atlas1_T2_ANTS.nii:	 36.2% -- replaced with atlas1_T2_ANTS.nii.gz
atlas2_brainmask_ANTS.nii:	 99.8% -- replaced with atlas2_brainmask_ANTS.nii.gz
atlas2_T1_ANTS.nii:	 39.0% -- replaced with atlas2_T1_ANTS.nii.gz
atlas2_T2_ANTS.nii:	 37.9% -- replaced with atlas2_T2_ANTS.nii.gz
atlas3_brainmask_ANTS.nii:	 99.8% -- replaced with atlas3_brainmask_ANTS.nii.gz
atlas3_T1_ANTS.nii:	 43.2% -- replaced with atlas3_T1_ANTS.nii.gz
atlas3_T2_ANTS.nii:	 42.7% -- replaced with atlas3_T2_ANTS.nii.gz
atlas4_brainmask_ANTS.nii:	 99.8% -- replaced with atlas4_brainmask_ANTS.nii.gz
atlas4_T1_ANTS.nii:	 43.8% -- replaced with atlas4_T1_ANTS.nii.gz
atlas4_T2_ANTS.nii:	 43.4% -- replaced with atlas4_T2_ANTS.nii.gz
t1.nii:	 40.3% -- replaced with t1.nii.gz
t2.nii:	 65.0% -- replaced with t2.nii.gz
Working di

Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas3_brainmask.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas3_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas3_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: NearestNeighborInterpolateImageFunction
Output warped image: atlas3_brainmask_ANTS.nii
AntsExample.sh t1.nii /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii fastfortesting atlas4_T1_ANTS.nii 6
Using double precision for computations.
Input scalar image: /home/sc-mlsg/MONSTR/TBI_atlas/atlas4_T1.nii
Reference image: t1.nii
The composite transform comprises the following transforms (in order): 
  1. atlas4_T1_ANTS0GenericAffine.mat (type = AffineTransform)
  2. atlas4_T1_ANTS1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: BSplineInterpolateImageFunction
Output warped image

Image Exception : #63 :: No image files match: t1_MultiModalStripMask_smooth
terminate called after throwing an instance of 'std::runtime_error'
  what():  No image files match: t1_MultiModalStripMask_smooth
./MONSTR.sh: line 578: 15419 Aborted                 (core dumped) fslmaths "$subs"_MultiModalStripMask_smooth.nii "$subs"_MultiModalStripMask_smooth.nii -odt char
/home/sc-mlsg/MONSTR/image_dilate t1_MultiModalStripMask_smooth.nii t1_MultiModalStripMask_smooth.nii 1
/home/sc-mlsg/MONSTR/image_dilate: error while loading shared libraries: libmwlaunchermain.so: cannot open shared object file: No such file or directory
invalid input dataset <t1_MultiModalStripMask_smooth.nii>
atlas1_brainmask_ANTS.nii:	 99.8% -- replaced with atlas1_brainmask_ANTS.nii.gz
atlas1_T1_ANTS.nii:	 37.1% -- replaced with atlas1_T1_ANTS.nii.gz
atlas1_T2_ANTS.nii:	 36.5% -- replaced with atlas1_T2_ANTS.nii.gz
atlas2_brainmask_ANTS.nii:	 99.8% -- replaced with atlas2_brainmask_ANTS.nii.gz
atlas2_T1_ANTS.nii:	 

In [20]:
# 2) apply skull mask
total = 0
error_dict = {}
for (path, dirs, files) in os.walk(sourceDir):
    try:
        if files != []:
            total += 1
            pt = path.split(os.path.sep)[-1]
            maskName = pt + '_brain_mask.nii.gz'
            mask = nib.load(os.path.join(brainmaskDir, maskName))

            for file in files:
                seq = find_seq(file)
                print("# {} Applying skull mask to {}...".format(total, file))
                outPath = get_outpath(strippedDir, pt, pt + "_{}_stripped.nii".format(seq))
                if outPath is None or 'label' in file:
                    continue
                
                # Reshape (in some cases data resolution is (x,y,z,1), reshape removes the 4th dimension
                file_nii = nib.load(os.path.join(path, file))
                file_data = file_nii.get_data()
                file_data = file_data.reshape(file_data.shape[0], file_data.shape[1], file_data.shape[2])

                masked_data = np.multiply(file_data, mask.get_data())
                masked_data = nib.Nifti1Image(masked_data, file_nii.affine, file_nii.header)
                nib.save(masked_data, outPath)
    except Exception as ex:
        print("Error:{}, filename:{}".format(ex.args, file))
        error_dict[pt] = ex.args

print("Finished!")
print(error_dict)

# 1 Applying skull mask to 001_T1C-label_resize.nii...
001_T1C_stripped.nii already exists
# 1 Applying skull mask to 001_T1C_registered.nii...
001_T1C_stripped.nii already exists
# 1 Applying skull mask to 001_T2_registered.nii...
001_T2_stripped.nii already exists
# 2 Applying skull mask to 002_T1C-label_resize.nii...
002_T1C_stripped.nii already exists
# 2 Applying skull mask to 002_T1C_registered.nii...
002_T1C_stripped.nii already exists
# 2 Applying skull mask to 002_T2_registered.nii...
002_T2_stripped.nii already exists
# 3 Applying skull mask to 003_T1C-label_resize.nii...
003_T1C_stripped.nii already exists
# 3 Applying skull mask to 003_T1C_registered.nii...
003_T1C_stripped.nii already exists
# 3 Applying skull mask to 003_T2_registered.nii...
003_T2_stripped.nii already exists
# 4 Applying skull mask to 006_T1C-label_resize.nii...
006_T1C_stripped.nii already exists
# 4 Applying skull mask to 006_T1C_registered.nii...
006_T1C_stripped.nii already exists
# 4 Applying skull 

# 37 Applying skull mask to 064_T1C-label_resize.nii...
064_T1C_stripped.nii already exists
# 37 Applying skull mask to 064_T1C_registered.nii...
064_T1C_stripped.nii already exists
# 37 Applying skull mask to 064_T2_registered.nii...
064_T2_stripped.nii already exists
# 38 Applying skull mask to 073_T1C-label_resize.nii...
073_T1C_stripped.nii already exists
# 38 Applying skull mask to 073_T1C_registered.nii...
073_T1C_stripped.nii already exists
# 38 Applying skull mask to 073_T2_registered.nii...
073_T2_stripped.nii already exists
# 39 Applying skull mask to 075_T1C-label_resize.nii...
075_T1C_stripped.nii already exists
# 39 Applying skull mask to 075_T1C_registered.nii...
075_T1C_stripped.nii already exists
# 39 Applying skull mask to 075_T2_registered.nii...
075_T2_stripped.nii already exists
# 40 Applying skull mask to 076_T1C-label_resize.nii...
076_T1C_stripped.nii already exists
# 40 Applying skull mask to 076_T1C_registered.nii...
076_T1C_stripped.nii already exists
# 40 App

/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


# 71 Applying skull mask to 129_T2_registered.nii...
# 72 Applying skull mask to 130_T1C-label_resize.nii...
130_T1C_stripped.nii already exists
# 72 Applying skull mask to 130_T1C_registered.nii...
130_T1C_stripped.nii already exists
# 72 Applying skull mask to 130_T2_registered.nii...
130_T2_stripped.nii already exists
# 73 Applying skull mask to 137_T1C-label_resize.nii...
# 73 Applying skull mask to 137_T1C_registered.nii...
# 73 Applying skull mask to 137_T2_registered.nii...
# 74 Applying skull mask to 139_T1C-label_resize.nii...
139_T1C_stripped.nii already exists
# 74 Applying skull mask to 139_T1C_registered.nii...
139_T1C_stripped.nii already exists
# 74 Applying skull mask to 139_T2_registered.nii...
139_T2_stripped.nii already exists
# 75 Applying skull mask to 141_T1C-label_resize.nii...
141_T1C_stripped.nii already exists
# 75 Applying skull mask to 141_T1C_registered.nii...
141_T1C_stripped.nii already exists
# 75 Applying skull mask to 141_T2_registered.nii...
141_T2_st

# 103 Applying skull mask to 219_T1C-label_resize.nii...
219_T1C_stripped.nii already exists
# 103 Applying skull mask to 219_T1C_registered.nii...
219_T1C_stripped.nii already exists
# 103 Applying skull mask to 219_T2_registered.nii...
219_T2_stripped.nii already exists
# 104 Applying skull mask to 221_T1C-label_resize.nii...
221_T1C_stripped.nii already exists
# 104 Applying skull mask to 221_T1C_registered.nii...
221_T1C_stripped.nii already exists
# 104 Applying skull mask to 221_T2_registered.nii...
221_T2_stripped.nii already exists
# 105 Applying skull mask to 222_T1C-label_resize.nii...
222_T1C_stripped.nii already exists
# 105 Applying skull mask to 222_T1C_registered.nii...
222_T1C_stripped.nii already exists
# 105 Applying skull mask to 222_T2_registered.nii...
222_T2_stripped.nii already exists
# 106 Applying skull mask to 223_T1C-label_resize.nii...
223_T1C_stripped.nii already exists
# 106 Applying skull mask to 223_T1C_registered.nii...
223_T1C_stripped.nii already exis

In [21]:
# 3) z-score normalization
total = 0
for (path, dirs, files) in os.walk(strippedDir):
    for file in files:
        try:
            total += 1
            print("Z-score normalization: #{}, {}".format(total, file))
            pt = path.split(os.path.sep)[-1]
            newfilename = file.split(".")[0] + "_znormed.nii.gz"
            znorm_outPath = get_outpath(znormedDir, pt, newfilename)
            if znorm_outPath is None:
                continue
            
            seq = find_seq(file)
            if seq == "ADC":
                print("ADC, copying file...")
                img_nii = nib.load(os.path.join(path, file))
                newfilename = file.split(".")[0] + ".nii.gz"
                nib.save(img_nii, os.path.join(znormedDir, pt, newfilename))
                continue
            img_nii = nib.load(os.path.join(path, file))
            img_data = img_nii.get_data()
            mean, std = np.mean(img_data), np.std(img_data)
            normalized = nib.Nifti1Image((img_data - mean) / std, img_nii.affine, img_nii.header)
            nib.save(normalized, znorm_outPath)
        except Exception as e:
            print(path, file)
            print(e)

Z-score normalization: #1, 001_T1C_stripped.nii
001_T1C_stripped_znormed.nii.gz already exists
Z-score normalization: #2, 001_T2_stripped.nii
001_T2_stripped_znormed.nii.gz already exists
Z-score normalization: #3, 002_T1C_stripped.nii
002_T1C_stripped_znormed.nii.gz already exists
Z-score normalization: #4, 002_T2_stripped.nii
002_T2_stripped_znormed.nii.gz already exists
Z-score normalization: #5, 003_T1C_stripped.nii
003_T1C_stripped_znormed.nii.gz already exists
Z-score normalization: #6, 003_T2_stripped.nii
003_T2_stripped_znormed.nii.gz already exists
Z-score normalization: #7, 006_T1C_stripped.nii
006_T1C_stripped_znormed.nii.gz already exists
Z-score normalization: #8, 006_T2_stripped.nii
006_T2_stripped_znormed.nii.gz already exists
Z-score normalization: #9, 008_T1C_stripped.nii
008_T1C_stripped_znormed.nii.gz already exists
Z-score normalization: #10, 008_T2_stripped.nii
008_T2_stripped_znormed.nii.gz already exists
Z-score normalization: #11, 009_T1C_stripped.nii
009_T1C_st

/home/sc-mlsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


Z-score normalization: #142, 129_T2_stripped.nii
Z-score normalization: #143, 130_T1C_stripped.nii
130_T1C_stripped_znormed.nii.gz already exists
Z-score normalization: #144, 130_T2_stripped.nii
130_T2_stripped_znormed.nii.gz already exists
Z-score normalization: #145, 137_T1C_stripped.nii
Z-score normalization: #146, 137_T2_stripped.nii
Z-score normalization: #147, 139_T1C_stripped.nii
139_T1C_stripped_znormed.nii.gz already exists
Z-score normalization: #148, 139_T2_stripped.nii
139_T2_stripped_znormed.nii.gz already exists
Z-score normalization: #149, 141_T1C_stripped.nii
141_T1C_stripped_znormed.nii.gz already exists
Z-score normalization: #150, 141_T2_stripped.nii
141_T2_stripped_znormed.nii.gz already exists
Z-score normalization: #151, 144_T1C_stripped.nii
144_T1C_stripped_znormed.nii.gz already exists
Z-score normalization: #152, 144_T2_stripped.nii
144_T2_stripped_znormed.nii.gz already exists
Z-score normalization: #153, 146_T1C_stripped.nii
146_T1C_stripped_znormed.nii.gz al

In [22]:
# 4) tumormask refine
tumormasks = glob(os.path.join(sourceDir, '**/*label*'))
total = 0

for filepath in tumormasks:
    try:
        total += 1
        file = os.path.basename(filepath)
        pt = file.split("_")[0]
        print(total, pt)
        if not os.path.exists(os.path.join(sourceDir, pt)):        
            os.makedirs(os.path.join(sourceDir, pt))

        # load tumor mask
        tumor_m = nib.load(filepath)

        # load brain mask
        brain_m = nib.load(os.path.join(brainmaskDir, "{}_brain_mask.nii.gz".format(pt)))

        masked_data = np.multiply(tumor_m.get_fdata(), brain_m.get_fdata())
        new_tumor_m = nib.Nifti1Image(masked_data, tumor_m.affine, tumor_m.header)

        nib.save(new_tumor_m, os.path.join(sourceDir, pt, file))
    except Exception as ex:
        print("Error:{}, filename:{}".format(ex.args, file))

1 001
2 002
3 003
4 006
5 008
6 009
7 010
8 011
9 013
10 014
11 017
12 018
13 019
14 020
15 021
16 022
17 023
18 026
19 027
20 028
21 029
22 032
23 034
24 035
25 037
26 038
27 039
28 046
29 048
30 049
31 050
32 054
33 055
34 059
35 060
36 061
37 064
38 073
39 075
40 076
41 077
42 078
43 079
44 080
45 085
46 088
47 089
48 090
49 091
50 092
51 094
52 096
53 097
54 099
55 100
56 102
57 103
58 105
59 106
60 107
61 111
62 113
63 115
64 117
65 119
66 121
67 125
68 126
69 127
70 128
71 129
72 130
73 137
74 139
75 141
76 144
77 146
78 147
79 150
80 152
81 153
82 154
83 155
84 157
85 160
86 161
87 201
88 202
89 204
90 205
91 206
92 207
93 208
94 209
95 210
96 211
97 212
98 213
99 214
100 215
101 217
102 218
103 219
104 221
105 222
106 223
107 224
108 225
109 226
110 227
111 228
112 229
113 230
114 231
115 232
116 233
117 234
118 235
119 236
120 237
121 238
122 240
123 033
124 083
125 124
126 203


In [23]:
# 5) cropping
total = 0
max_height, max_width, max_depth = 0, 0, 0

# Loop through each folder with `.nii.gz` files.
for (path, dirs, files) in os.walk(znormedDir):
    total +=1
    if dirs == []:
        pt = path.split(os.path.sep)[-1]
        # load tumor mask
        tumormaskName = '{}_T1C-label_resize.nii'.format(pt)
        files.append(tumormaskName)
        print(files)
        
        brain_arr = nib.load(os.path.join(path, files[1])).get_fdata()
        print(brain_arr.shape)
        # Determine smallest bounding box.
        h_min, h_max = np.where(np.any(brain_arr>0.0, axis=(1, 2)))[0][[0, -1]]
        w_min, w_max = np.where(np.any(brain_arr>0.0, axis=(0, 2)))[0][[0, -1]]
        d_min, d_max = np.where(np.any(brain_arr>0.0, axis=(0, 1)))[0][[0, -1]]
                
        for file in files:
            outPath = get_outpath(croppedDir, pt, "{}_{}_cropped.nii".format(pt, file.split("_")[1]))
            if outPath is None:
                continue
            if 'label' in file:
                brain_nii = nib.load(os.path.join(sourceDir, pt, file))
            else:
                brain_nii = nib.load(os.path.join(path, file))
            brain_arr = brain_nii.get_fdata()

            # Crop to minimal size for all examples.
            cropped_arr = brain_arr[h_min:h_max, w_min:w_max, d_min:d_max]
            #print(x)

            # save
            cropped_nii = nib.Nifti1Image(cropped_arr, brain_nii.affine, brain_nii.header)
            nib.save(cropped_nii, outPath)
            
            print(cropped_arr.shape)
        max_height = max(max_height, cropped_arr.shape[0])
        max_width = max(max_width, cropped_arr.shape[1])
        max_depth = max(max_depth, cropped_arr.shape[2])
print(max_height, max_width, max_depth)

['001_T1C_stripped_znormed.nii.gz', '001_T2_stripped_znormed.nii.gz', '001_T1C-label_resize.nii']
(256, 256, 256)
(132, 153, 138)
(132, 153, 138)
(132, 153, 138)
['002_T1C_stripped_znormed.nii.gz', '002_T2_stripped_znormed.nii.gz', '002_T1C-label_resize.nii']
(256, 256, 256)
(131, 155, 138)
(131, 155, 138)
(131, 155, 138)
['003_T1C_stripped_znormed.nii.gz', '003_T2_stripped_znormed.nii.gz', '003_T1C-label_resize.nii']
(256, 256, 256)
(128, 155, 141)
(128, 155, 141)
(128, 155, 141)
['006_T1C_stripped_znormed.nii.gz', '006_T2_stripped_znormed.nii.gz', '006_T1C-label_resize.nii']
(256, 256, 256)
(143, 148, 137)
(143, 148, 137)
(143, 148, 137)
['008_T1C_stripped_znormed.nii.gz', '008_T2_stripped_znormed.nii.gz', '008_T1C-label_resize.nii']
(256, 256, 256)
(136, 150, 143)
(136, 150, 143)
(136, 150, 143)
['009_T1C_stripped_znormed.nii.gz', '009_T2_stripped_znormed.nii.gz', '009_T1C-label_resize.nii']
(256, 256, 192)
(142, 157, 154)
(142, 157, 154)
(142, 157, 154)
['010_T1C_stripped_znormed.n

(256, 256, 256)
(131, 156, 144)
(131, 156, 144)
(131, 156, 144)
['096_T1C_stripped_znormed.nii.gz', '096_T2_stripped_znormed.nii.gz', '096_T1C-label_resize.nii']
(256, 256, 256)
(131, 143, 139)
(131, 143, 139)
(131, 143, 139)
['097_T1C_stripped_znormed.nii.gz', '097_T2_stripped_znormed.nii.gz', '097_T1C-label_resize.nii']
(256, 256, 256)
(131, 160, 142)
(131, 160, 142)
(131, 160, 142)
['099_T1C_stripped_znormed.nii.gz', '099_T2_stripped_znormed.nii.gz', '099_T1C-label_resize.nii']
(256, 256, 256)
(131, 159, 144)
(131, 159, 144)
(131, 159, 144)
['100_T1C_stripped_znormed.nii.gz', '100_T2_stripped_znormed.nii.gz', '100_T1C-label_resize.nii']
(256, 256, 256)
(131, 162, 140)
(131, 162, 140)
(131, 162, 140)
['102_T1C_stripped_znormed.nii.gz', '102_T2_stripped_znormed.nii.gz', '102_T1C-label_resize.nii']
(256, 256, 256)
(147, 158, 141)
(147, 158, 141)
(147, 158, 141)
['103_T1C_stripped_znormed.nii.gz', '103_T2_stripped_znormed.nii.gz', '103_T1C-label_resize.nii']
(256, 256, 160)
(136, 155, 1

(256, 256, 256)
(142, 158, 151)
(142, 158, 151)
(142, 158, 151)
['219_T1C_stripped_znormed.nii.gz', '219_T2_stripped_znormed.nii.gz', '219_T1C-label_resize.nii']
(256, 256, 176)
(135, 150, 142)
(135, 150, 142)
(135, 150, 142)
['221_T1C_stripped_znormed.nii.gz', '221_T2_stripped_znormed.nii.gz', '221_T1C-label_resize.nii']
(256, 256, 256)
(146, 166, 146)
(146, 166, 146)
(146, 166, 146)
['222_T1C_stripped_znormed.nii.gz', '222_T2_stripped_znormed.nii.gz', '222_T1C-label_resize.nii']
(256, 256, 160)
(129, 159, 140)
(129, 159, 140)
(129, 159, 140)
['223_T1C_stripped_znormed.nii.gz', '223_T2_stripped_znormed.nii.gz', '223_T1C-label_resize.nii']
(256, 256, 256)
(135, 159, 137)
(135, 159, 137)
(135, 159, 137)
['224_T1C_stripped_znormed.nii.gz', '224_T2_stripped_znormed.nii.gz', '224_T1C-label_resize.nii']
(256, 256, 256)
(136, 165, 143)
(136, 165, 143)
(136, 165, 143)
['225_T1C_stripped_znormed.nii.gz', '225_T2_stripped_znormed.nii.gz', '225_T1C-label_resize.nii']
(256, 256, 256)
(135, 148, 1

In [26]:
# 6) padding

total = 0
tar_height, tar_width, tar_depth = 220, 220, 220

# Loop through each folder with `.nii.gz` files.
for (path, dirs, files) in os.walk(croppedDir):
    total +=1
    if dirs == []:
        pt = path.split(os.path.sep)[-1]
        print(files)
        
        brain_arr = nib.load(os.path.join(path, files[1])).get_fdata()
        print(brain_arr.shape)
        pad1 = (tar_height - brain_arr.shape[0]) / 2
        pad2 = (tar_width - brain_arr.shape[1]) / 2
        pad3 = (tar_depth - brain_arr.shape[2]) / 2
        
        for file in files:
            outPath = get_outpath(paddedDir, pt, "{}_{}_resized.nii".format(pt, file.split("_")[1]))
            if outPath is None:
                continue
            brain_nii = nib.load(os.path.join(path, file))
            brain_arr = brain_nii.get_fdata()

            # pad
            padded_arr = np.pad(brain_arr, ((np.int(np.floor(pad1)),np.int(np.ceil(pad1))),(np.int(np.floor(pad2)),np.int(np.ceil(pad2))),(np.int(np.floor(pad3)),np.int(np.ceil(pad3)))))
            #print(x)

            # save
            paddeded_nii = nib.Nifti1Image(padded_arr, brain_nii.affine, brain_nii.header)
            nib.save(paddeded_nii, outPath)
            
            print(padded_arr.shape)

['001_T1C-label_cropped.nii', '001_T1C_cropped.nii', '001_T2_cropped.nii']
(132, 153, 138)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['002_T1C-label_cropped.nii', '002_T1C_cropped.nii', '002_T2_cropped.nii']
(131, 155, 138)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['003_T1C-label_cropped.nii', '003_T1C_cropped.nii', '003_T2_cropped.nii']
(128, 155, 141)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['006_T1C-label_cropped.nii', '006_T1C_cropped.nii', '006_T2_cropped.nii']
(143, 148, 137)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['008_T1C-label_cropped.nii', '008_T1C_cropped.nii', '008_T2_cropped.nii']
(136, 150, 143)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['009_T1C-label_cropped.nii', '009_T1C_cropped.nii', '009_T2_cropped.nii']
(142, 157, 154)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['010_T1C-label_cropped.nii', '010_T1C_cropped.nii', '010_T2_cropped.nii']
(136, 159, 145)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['011_T1C-label_cropped.nii

(141, 161, 138)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['111_T1C-label_cropped.nii', '111_T1C_cropped.nii', '111_T2_cropped.nii']
(145, 149, 160)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['113_T1C-label_cropped.nii', '113_T1C_cropped.nii', '113_T2_cropped.nii']
(134, 154, 143)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['115_T1C-label_cropped.nii', '115_T1C_cropped.nii', '115_T2_cropped.nii']
(133, 151, 131)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['117_T1C-label_cropped.nii', '117_T1C_cropped.nii', '117_T2_cropped.nii']
(147, 168, 149)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['119_T1C-label_cropped.nii', '119_T1C_cropped.nii', '119_T2_cropped.nii']
(135, 146, 145)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['121_T1C-label_cropped.nii', '121_T1C_cropped.nii', '121_T2_cropped.nii']
(142, 149, 135)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['124_T1C-label_cropped.nii', '124_T1C_cropped.nii', '124_T2_cropped.nii']
(135, 156, 137)
(220, 220, 

(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['237_T1C-label_cropped.nii', '237_T1CE_cropped.nii', '237_T2_cropped.nii']
(131, 156, 145)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['238_T1C-label_cropped.nii', '238_T1C_cropped.nii', '238_T2_cropped.nii']
(134, 153, 145)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['240_T1C-label_cropped.nii', '240_T1C_cropped.nii', '240_T2_cropped.nii']
(140, 148, 137)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['034_T1C-label_cropped.nii', '034_T1C_cropped.nii', '034_T2_cropped.nii']
(129, 162, 142)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['088_T1C-label_cropped.nii', '088_T1C_cropped.nii', '088_T2_cropped.nii']
(143, 163, 151)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['127_T1C-label_cropped.nii', '127_T1C_cropped.nii', '127_T2_cropped.nii']
(138, 153, 147)
(220, 220, 220)
(220, 220, 220)
(220, 220, 220)
['206_T1C-label_cropped.nii', '206_T1C_cropped.nii', '206_T2_cropped.nii']
(140, 168, 152)
(220, 220, 220)
(220, 220,